# Naver API를 사용하여 Naver news crawling

# 1. 뉴스링크가져오기 (keyword = 배양육)

In [1]:
#뉴스링크가져오기
import requests
import time
from datetime import datetime
import pickle

def get_naver_news_search(keyword, start = 1, display = 100):
    
    base_url = "https://openapi.naver.com/v1/search/news?query={}&start={}&display={}"
    
    headers = {"X-Naver-Client-Id" : "your id",
                              "X-Naver-Client-Secret" : "password"}
    
    news_links = []
    
    while start <= 1000:
        url = base_url.format(keyword, start, display)
        
        res = requests.get(url, headers = headers)
        if res.status_code == 200:
            result = res.json()
            items = result['items']
            #네이버 링크들 추출
            for item in items:
                link = item['link']
                if 'news.naver.com' in link:
                    news_links.append(link)
            start = start + display
            time.sleep(1)
            #네이버 open api 요청은 초당 10회까지 호출가능하기 때문에 시간차를 둠
        else:
            raise Exception('검색실패 : {}'.format(res.status_code))
        
    
    curr = datetime.now().strftime('%Y-%m-%d')
    filename = "네이버뉴스검색_{}_{}.pkl".format(keyword, curr)
    with open (filename, "wb") as f:
        pickle.dump(news_links, f)


In [2]:
#위의 뉴스링크 가져오는 함수 실행
get_naver_news_search ("배양육")

# 2. 뉴스링크로부터 기사내용 가져오기

In [3]:
#1개의 네이버 뉴스링크로부터 1개의 기사내용 가져오는 함수
from bs4 import BeautifulSoup
import requests

def get_naver_news(url):
    
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    
    #요청
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        #응답을 받은 url - res.url
        if 'news.naver.com' not in res.url:
            raise Exception('{}은 요청할 수 없는 url입니다.'.format(res.url))
        
        soup = BeautifulSoup(res.text)
        title = soup.select_one('h3#articleTitle').text.strip()
        input_date = soup.select_one('span.t11').text.strip()
        article = soup.select_one('div#articleBodyContents').text.strip()

        return title, input_date, article
    
    else:
        raise Exception("요청실패 : {}".format(res.status_code))

In [5]:
#위의 함수를 이용하여 기사내용 크롤링

import pickle
import pandas as pd

curr = datetime.now().strftime('%Y-%m-%d')

with open("네이버뉴스검색_{}_{}.pkl".format("배양육", curr), 'rb') as f:
    l = pickle.load(f)

result = []
error_cnt = 0
for url in l:
    try:
        info = get_naver_news(url)
        result.append(info)
    except:
        error_cnt += 1
        
#파일로 저장
print('error_cnt : {}'.format(error_cnt))
df = pd.DataFrame(result, columns = ["title", "date", "article"])
df.to_csv("cultured_meat.csv", index = False)
print('종료')

error_cnt : 1
종료
